# 颜色识别追逐小车

本notebook实现了一个追逐一个红色物体的小车。图像处理使用了PL部分由HLS编写的的color_detect模块，它返回物体的坐标。这个notebook将演示如何将图像处理和运动控制相结合。

## 导入所需文件并对PL编程

In [1]:
import numpy as np
from PYNQ_Car.CarOverlay.CarOverlay import CarOverlay
from pynq import GPIO
from PIL import Image
import IPython
import time
from pynq.xlnk import Xlnk
from pynq.lib.video.common import VideoMode
import cv2
xlnk = Xlnk()
overlay = CarOverlay('/usr/local/lib/python3.6/dist-packages/PYNQ_Car/Overlay/Car.bit')

## 初始化microblaze

In [2]:
arduino = overlay.Arduino
arduino.load('bimotor')
arduino.set_iic_pins('SDA','SCL') #SDA  SCL
arduino.set_motor_pins('D11','D5','D6','D3')
arduino.set_servo_pin('D9')
arduino.init()
arduino.set_motor_dir(1,0)
arduino.set_encoder_dir(1,0)

## 初始化图像通路

图像的输出将使用VDMA实现，这里使用pynq的默认驱动。执行完下列代码后HDMI接口将输出摄像头读取到的图像。

In [3]:
OV5640 = overlay.OV5640
video_proc = overlay.video_proc
hdmi_out = overlay.video_output.axi_vdma.writechannel
hdmi_out.mode = VideoMode(1280,720,32)
OV5640.set_rgba()
OV5640.readchannel.tie(hdmi_out)
hdmi_out.start()

## 启动颜色识别和追踪

HDMI将输出识别结果，可以用于调试参数。舵机会随着识别出来的横坐标转动。

In [4]:
x0 = 0
y0 = 0
x1 = 0
y1 = 0
H_low  = 0
H_high = 17
S_low = 60
S_high = 255
V_low = 50
V_high = 255
servo_ctr_p = 0.07
arduino.set_velocity(300)
angle = 0.0
arduino.set_servo_angle(angle)
while True:
    src = OV5640.get_cam_frame()
    posX,posY,widthX,widthY = video_proc.ColorDetect(src,H_low,H_high,S_low,S_high,V_low,V_high)
    if (widthX > 50 and widthY > 50):
        x0 = posX - (widthX>>1)
        y0 = posY - (widthY>>1)
        x1 = posX + (widthX>>1)
        y1 = posY + (widthY>>1)
        angle = float((posX-640)*servo_ctr_p)
    cv2.rectangle(src,(x0,y0),(x1,y1),(0,255,0,0),2)
    arduino.set_servo_angle(angle)
    hdmi_out.writeframe(src)

KeyboardInterrupt: 

### 结束并停止一切功能

In [5]:
arduino.set_velocity(0)
arduino.set_servo_angle(0.0)
hdmi_out.stop()
OV5640.stop()
video_proc.stop()